In [1]:
#!pip install rapidfuzz # Нужно для Colab
import pandas as pd
import folium
import json
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from rapidfuzz import fuzz, process
from datetime import datetime
from functools import lru_cache
import requests
import os
from folium.features import CustomIcon
from folium.plugins import GroupedLayerControl
from matplotlib.patches import Wedge
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# Словарь для хранения важных рабочих английских названий столбцов
column_names = {
    "region": "region",
    "year": "year",
    "total_population": "total_population"
}

available_data = []
combined_data_df = None
geojson_data = None
map_parameters = {}
header_mapping = {}

# Функция для загрузки файлов
def load_data_files(source_choice, geojson_url=None, csv_url=None):
    default_geojson_path = 'Voronezh_region.geojson'
    default_csv_path = 'combined_table.csv'
    custom_folder = 'custom_files'
    default_geojson_url = 'https://raw.githubusercontent.com/Dimatron74/foliumVRN/refs/heads/main/json/Voronezh_region.geojson'
    default_csv_url = 'https://raw.githubusercontent.com/Dimatron74/foliumVRN/refs/heads/main/combined_table.csv'

    if not os.path.exists(custom_folder):
        os.makedirs(custom_folder)

    def download_from_url(file_path, url):
        if not os.path.exists(file_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"Файл {file_path} успешно загружен с {url}")
            else:
                print(f"Ошибка при загрузке {file_path}: HTTP {response.status_code}")
        else:
            print(f"Файл {file_path} уже существует.")

    if source_choice == '1':
        # Загрузка стандартных файлов
        download_from_url(default_geojson_path, default_geojson_url)
        download_from_url(default_csv_path, default_csv_url)
        return default_geojson_path, default_csv_path
    elif source_choice == '2':
        # Загрузка пользовательских файлов
        custom_geojson_path = os.path.join(custom_folder, 'custom_Voronezh_region.geojson')
        custom_csv_path = os.path.join(custom_folder, 'custom_combined_table.csv')
        download_from_url(custom_geojson_path, geojson_url)
        download_from_url(custom_csv_path, csv_url)
        return custom_geojson_path, custom_csv_path
    else:
        # Загрузка стандартных файлов по умолчанию
        download_from_url(default_geojson_path, default_geojson_url)
        download_from_url(default_csv_path, default_csv_url)
        return default_geojson_path, default_csv_path

# Чтение данных
def read_data(geojson_path, csv_path):
    global combined_data_df, geojson_data
    # Чтение CSV
    combined_data_df = pd.read_csv(csv_path, sep=None, encoding='utf-8-sig', header=[0, 1])
    english_headers = combined_data_df.columns.get_level_values(0).str.lower()
    russian_headers = combined_data_df.columns.get_level_values(1)
    global header_mapping
    header_mapping = dict(zip(english_headers, russian_headers))
    combined_data_df.columns = english_headers
    combined_data_df[column_names["region"]] = combined_data_df[column_names["region"]].str.strip().str.lower().str.replace('ё', 'е')
    global available_data
    available_data = [col for col in combined_data_df.columns if col not in [column_names["region"], column_names["year"]]]
    # Чтение GeoJSON
    with open(geojson_path, 'r', encoding='utf-8') as f:
        geojson_data = json.load(f)

# Интерфейс для выбора источника данных
def create_data_source_interface():
    source_selection = widgets.RadioButtons(
        options=[('Использовать стандартные файлы', '1'), ('Загрузить свои файлы по ссылкам', '2')],
        description='Выбор источника:',
        disabled=False
    )

    custom_geojson_url = widgets.Text(description="GeoJSON URL:", value='')
    custom_csv_url = widgets.Text(description="CSV URL:", value='')

    load_button = widgets.Button(description="Загрузить данные")
    output = widgets.Output()

    data_source_box = widgets.VBox([
        source_selection,
        widgets.HTML(value="<b>Шаг 1: Выбор источника данных</b><br>Выберите способ загрузки данных."),
        widgets.HBox([custom_geojson_url, custom_csv_url], layout=widgets.Layout(display='none'))
    ])

    def toggle_url_fields(change):
        if change['new'] == '2':
            data_source_box.children[-1].layout.display = 'flex'
        else:
            data_source_box.children[-1].layout.display = 'none'

    source_selection.observe(toggle_url_fields, names='value')

    def load_data(b):
        with output:
            output.clear_output()
            try:
                source_choice = source_selection.value
                geojson_url = custom_geojson_url.value.strip() if source_choice == '2' else None
                csv_url = custom_csv_url.value.strip() if source_choice == '2' else None
                geojson_path, csv_path = load_data_files(source_choice, geojson_url, csv_url)
                read_data(geojson_path, csv_path)
                print("Данные успешно загружены!")
                create_map_parameters_interface()
            except Exception as e:
                print("Ошибка при загрузке данных:", str(e))

    load_button.on_click(load_data)

    display(
        widgets.VBox([
            data_source_box,
            load_button,
            output
        ])
    )

# Создание интерфейса для выбора параметров карты
def create_map_parameters_interface():
    map_type_selection = widgets.RadioButtons(
        options=[('GeoJSON', 'geojson'), ('Населенные пункты (OpenStreetMap)', 'osm')],
        description='Тип данных:',
        disabled=False
    )
    
    lat_input = widgets.FloatText(value=51.67, description='Широта:', disabled=False)
    lon_input = widgets.FloatText(value=39.18, description='Долгота:', disabled=False)
    zoom_input = widgets.IntSlider(value=8, min=1, max=18, step=1, description='Zoom:')
    
    column_checkboxes = []
    for column in available_data:
        checkbox = widgets.Checkbox(value=True, description=get_russian_name(column))
        column_checkboxes.append(checkbox)
    columns_box = widgets.VBox(column_checkboxes)
    
    years = combined_data_df[column_names["year"]].unique()
    year_selection = widgets.Dropdown(
        options=sorted(years),
        value=max(years),
        description='Год:',
        disabled=False,
    )
    
    # Добавление выбора типа маркеров
    marker_type_selection = widgets.RadioButtons(
        options=[('Стандартные', 'standard'), ('Столбчатая диаграмма', 'bar'), 
                 ('Круговая диаграмма', 'pie'), ('Круговые маркеры', 'circle')],
        description='Тип маркеров:',
        disabled=False,
        layout=widgets.Layout(display='none')
    )
    
    status_label = widgets.Label(value="")
    
    confirm_button = widgets.Button(description="Подтвердить параметры")
    
    def toggle_marker_type_visibility(change):
        if change['new'] == 'osm':
            marker_type_selection.layout.display = 'flex'
        else:
            marker_type_selection.layout.display = 'none'
    
    map_type_selection.observe(toggle_marker_type_visibility, names='value')
    
    def confirm_parameters(b):
        selected_map_type = map_type_selection.value
        selected_lat = float(lat_input.value)
        selected_lon = float(lon_input.value)
        selected_zoom = int(zoom_input.value)
        selected_columns = [cb.description for cb in column_checkboxes if cb.value]
        selected_year = int(year_selection.value)
        selected_marker_type = marker_type_selection.value
        
        print("Выбранные параметры:")
        print(f"- Тип данных: {selected_map_type}")
        print(f"- Координаты: ({selected_lat}, {selected_lon})")
        print(f"- Zoom: {selected_zoom}")
        print(f"- Выбранные столбцы: {', '.join(selected_columns) or 'Нет'}")
        print(f"- Год: {selected_year}")
        print(f"- Тип маркеров: {selected_marker_type}")
        
        global map_parameters
        map_parameters = {
            "map_type": selected_map_type,
            "center": [selected_lat, selected_lon],
            "zoom": selected_zoom,
            "columns": selected_columns,
            "year": selected_year,
            "marker_type": selected_marker_type
        }
        
        # Показываем статус создания карты
        status_label.value = "Создание карты... Пожалуйста, подождите."
        
        # Переходим к созданию карты
        create_final_map()
        
        # Обновляем статус после сохранения карты
        status_label.value = f"Карта успешно сохранена по пути: final_map.html"
    
    confirm_button.on_click(confirm_parameters)
    
    display(
        widgets.VBox([
            widgets.HTML(value="<b>Шаг 1: Выбор типа данных для построения карты</b><br>Выберите источник данных для карты."),
            map_type_selection,
            marker_type_selection,
            widgets.HTML(value="<b>Шаг 2: Установка геокоординат</b><br>Введите начальные координаты и масштаб карты."),
            widgets.HBox([lat_input, lon_input]),
            zoom_input,
            widgets.HTML(value="<b>Шаг 3: Выбор столбцов для отображения</b><br>Выберите данные, которые будут отображаться на карте."),
            columns_box,
            widgets.HTML(value="<b>Шаг 4: Выбор года отображения</b><br>Выберите год, данные за который будут показаны."),
            year_selection,
            confirm_button,
            status_label
        ])
    )

# Функции обработки данных
def get_russian_name(english_name):
    return header_mapping.get(english_name, english_name)

# Функция для получения данных о населённых пунктах с OpenStreetMap
def fetch_settlements_from_overpass():
    """Запрашивает данные о населённых пунктах Воронежской области через Overpass API."""
    print("Получение данных о населённых пунктах с OpenStreetMap...")
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = """
    [out:json][timeout:25];
    area["name"="Воронежская область"]["admin_level"="4"]->.region;
    (
      node["place"](area.region);
    );
    out body;
    >;
    out skel qt;
    """
    response = requests.post(overpass_url, data=overpass_query)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка при запросе к Overpass API: {response.status_code}")
        return None

# Функция для обработки данных о населённых пунктах
def process_settlements_data(settlements_data, combined_data_df):
    """Обрабатывает данные о населённых пунктах, добавляя население из CSV."""
    settlements = []
    for element in settlements_data.get("elements", []):
        if element.get("type") == "node" and "tags" in element:
            tags = element["tags"]
            settlement_name = tags.get("name", "").strip().lower().replace('ё', 'е')
            lat = element.get("lat")
            lon = element.get("lon")
            match = find_matching_region(settlement_name, combined_data_df[column_names["region"]].unique())
            if match:
                region_data = combined_data_df[combined_data_df[column_names["region"]] == match]
                population = None
                if not region_data.empty:
                    grouped_data = group_columns_by_year(combined_data_df, match)
                    current_year = datetime.now().year
                    population = get_closest_year_value(grouped_data, column_names["total_population"], current_year)
                settlements.append({
                    "name": settlement_name.capitalize(),
                    "lat": lat,
                    "lon": lon,
                    "population": int(population) if population is not None else None
                })
    return settlements


# Функции создания диаграмм для маркеров населённых пунктов
def create_bar_chart_icon(settlement, grouped_data, data_type):
    data_values = grouped_data.get(data_type, {})
    if not data_values:
        years = [str(datetime.now().year)]
        values = [0]
    else:
        years = sorted(data_values.keys())
        values = [float(data_values[year]) for year in years]
    
    plt.figure(figsize=(2, 1.5))
    if len(years) == 1:
        positions = [-1, 0, 1]
        bar_values = [0, values[0], 0]
        colors = ['white', 'gray' if values[0] > 0 else 'blue', 'white']
        plt.bar(positions, bar_values, width=0.4, color=colors, alpha=0.6)
        plt.xticks([0], years, fontsize=6, rotation=45)
    else:
        plt.bar(range(len(years)), values, width=0.4,
                color=['blue', 'green', 'red', 'orange', 'purple'][:len(years)], alpha=0.6)
        plt.xticks(range(len(years)), years, fontsize=6, rotation=45)
    
    plt.yticks(fontsize=6)
    plt.axis('off')
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight', transparent=True)
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    plt.close()
    return CustomIcon(f"data:image/png;base64,{image_base64}", icon_size=(50, 50))

def create_pie_chart_icon(settlement, grouped_data, data_type):
    data_values = grouped_data.get(data_type, {})
    if not data_values:
        years = [str(datetime.now().year)]
        values = [0]
    else:
        years = sorted(data_values.keys())
        values = [float(data_values[year]) for year in years]
    
    total = sum(values) if values else 1
    percentages = [value / total * 100 if total > 0 else 0 for value in values]
    color_list = ['blue', 'green', 'red', 'orange', 'purple']
    
    fig, ax = plt.subplots(figsize=(1, 1), dpi=100)
    fig.patch.set_alpha(0)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.axis('off')
    start_angle = 0
    colors = ['gray'] if len(years) == 1 and values[0] > 0 else color_list[:len(years)]
    
    for i, percentage in enumerate(percentages):
        wedge = Wedge((0, 0), 1, start_angle, start_angle + percentage * 3.6,
                      facecolor=colors[i], edgecolor='black', alpha=0.5)
        ax.add_patch(wedge)
        start_angle += percentage * 3.6
    
    fig.canvas.draw()
    image = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig)
    img = Image.fromarray(image)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return CustomIcon(f"data:image/png;base64,{img_base64}", icon_size=(50, 50))

# Функции создания маркеров населённых пунктов
def add_bar_chart_markers(settlements, layer, data_type):
    for settlement in settlements:
        region_name, grouped_data, value = get_settlement_data(settlement, data_type)
        if not region_name:
            print(f"Соответствие не найдено для {settlement['name']}")
            continue
        
        data_values = grouped_data.get(data_type, {})
        if not data_values:
            years = [str(datetime.now().year)]
            values = [0]
        else:
            years = sorted(data_values.keys())
            values = [float(data_values[year]) for year in years]
        
        total = sum(values) if values else 1
        tooltip_text = f"{settlement['name']}<br>"
        for year, val in zip(years, values):
            percentage = val / total * 100 if total > 0 else 0
            tooltip_text += f"{year}: {int(val)} ({percentage:.1f}%)<br>"
        
        icon = create_bar_chart_icon(settlement, grouped_data, data_type)
        if icon:
            popup_html = f"<b>Населённый пункт:</b> {settlement['name']}<br>"
            if value is not None:
                popup_html += f"<b>{get_russian_name(data_type)}:</b> {value}"
            folium.Marker(
                location=[settlement["lat"], settlement["lon"]],
                icon=icon,
                popup=folium.Popup(popup_html, max_width=300),
                tooltip=folium.Tooltip(tooltip_text)
            ).add_to(layer)

def add_pie_chart_markers(settlements, layer, data_type):
    for settlement in settlements:
        region_name, grouped_data, value = get_settlement_data(settlement, data_type)
        if not region_name:
            print(f"Соответствие не найдено для {settlement['name']}")
            continue
        
        data_values = grouped_data.get(data_type, {})
        if not data_values:
            years = [str(datetime.now().year)]
            values = [0]
        else:
            years = sorted(data_values.keys())
            values = [float(data_values[year]) for year in years]
        
        total = sum(values) if values else 1
        tooltip_text = f"{settlement['name']}<br>"
        for year, val in zip(years, values):
            percentage = val / total * 100 if total > 0 else 0
            tooltip_text += f"{year}: {int(val)} ({percentage:.1f}%)<br>"
        
        icon = create_pie_chart_icon(settlement, grouped_data, data_type)
        if icon:
            popup_html = f"<b>Населённый пункт:</b> {settlement['name']}<br>"
            if value is not None:
                popup_html += f"<b>{get_russian_name(data_type)}:</b> {value}"
            folium.Marker(
                location=[settlement["lat"], settlement["lon"]],
                icon=icon,
                popup=folium.Popup(popup_html, max_width=300),
                tooltip=folium.Tooltip(tooltip_text)
            ).add_to(layer)

def add_standard_markers(settlements, layer, data_type):
    for settlement in settlements:
        region_name, grouped_data, value = get_settlement_data(settlement, data_type)
        popup_html = f"<b>Населённый пункт:</b> {settlement['name']}<br>"
        tooltip_text = settlement['name']
        if value is not None:
            popup_html += f"<b>{get_russian_name(data_type)}:</b> {value}"
            tooltip_text += f" ({value})"
        folium.Marker(
            location=[settlement["lat"], settlement["lon"]],
            tooltip=tooltip_text,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(layer)

def add_circle_markers(settlements, layer, data_type):
    for settlement in settlements:
        region_name, grouped_data, value = get_settlement_data(settlement, data_type)
        radius = 5 if value is None else min(max(value / 1000, 5), 20)
        popup_html = f"<b>Населённый пункт:</b> {settlement['name']}<br>"
        tooltip_text = settlement['name']
        if value is not None:
            popup_html += f"<b>{get_russian_name(data_type)}:</b> {value}"
            tooltip_text += f" ({value})"
        folium.CircleMarker(
            location=[settlement["lat"], settlement["lon"]],
            radius=radius,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=tooltip_text
        ).add_to(layer)


# Обновленная функция создания маркеров
def add_markers_based_on_type(settlements, layer, data_type, marker_type):
    if marker_type == 'bar':
        add_bar_chart_markers(settlements, layer, data_type)
    elif marker_type == 'pie':
        add_pie_chart_markers(settlements, layer, data_type)
    elif marker_type == 'circle':
        add_circle_markers(settlements, layer, data_type)
    else:
        add_standard_markers(settlements, layer, data_type)





def group_columns_by_year(df, region_name):
    region_data = df[df[column_names["region"]] == region_name]
    grouped_data = {}
    data_columns = [col for col in df.columns if col not in [column_names["region"], column_names["year"]]]
    for col in data_columns:
        grouped_data[col] = {}
        for _, row in region_data.iterrows():
            year = str(row[column_names["year"]])
            value = row[col]
            grouped_data[col][year] = value
    return grouped_data

def get_closest_year_value(grouped_data, column, current_year):
    if column not in grouped_data or not grouped_data[column]:
        return None
    years = list(grouped_data[column].keys())
    values = list(grouped_data[column].values())
    if not years:
        return None
    closest_year = min(years, key=lambda y: abs(int(y) - current_year))
    return float(values[years.index(closest_year)])

def find_matching_region(target, candidates):
    target_normalized = target.strip().lower().replace('ё', 'е')
    if target_normalized in candidates:
        return target_normalized
    match = process.extractOne(target_normalized, candidates, scorer=fuzz.token_set_ratio, score_cutoff=85)
    if match:
        best_match = match[0]
        if any(best_match != candidate and best_match in candidate for candidate in candidates):
            return None
        return best_match
    return None

def create_chart(region_name, grouped_data, column):
    group_name = get_russian_name(column)
    if group_name == column:
        print(f"Предупреждение: Русское название не найдено для '{column}'.")
    if column not in grouped_data or not grouped_data[column]:
        return None
    years = [int(year) for year in grouped_data[column].keys()]
    values = [float(value) for value in grouped_data[column].values()]
    if not years or not values:
        return None
    plt.figure(figsize=(3, 2))
    bars = plt.bar(years, values, color=['blue', 'green', 'red'], alpha=0.6, label=group_name)
    plt.plot(years, values, color='black', marker='o', linewidth=2, label='Тренд')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.5, f"{yval:.1f}", ha='center', va='bottom', fontsize=8)
    plt.title(f'{group_name}\n{region_name}', fontsize=10)
    plt.xlabel('Год', fontsize=8)
    plt.ylabel(group_name, fontsize=8)
    plt.xticks(years, fontsize=8)
    plt.yticks(fontsize=8)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend(fontsize=8)
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    plt.close()
    return f'<img src="data:image/png;base64,{image_base64}" style="max-width: 100%; height: auto;">'

# Функция стиля для GeoJSON
def style_function(feature):
    district_name = feature['properties']['district'].lower().replace('ё', 'е')
    match, population, chart_html = process_district_data(district_name)
    if match:
        popup_html = create_popup_html(district_name.capitalize(), chart_html)
        feature['properties']['popup_html'] = popup_html
        if population:
            feature['properties']['population'] = int(population)
            if population < 35000:
                color = 'green'
            elif population < 500000:
                color = 'orange'
            elif population > 500000:
                color = 'red'
            else:
                color = 'grey'
        else:
            feature['properties']['population'] = "Данные отсутствуют"
            color = 'grey'
    else:
        color = 'grey'
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.4
    }

# Функция для создания единого HTML для всплывающего окна
def create_popup_html(district_name, chart_html):
    html_template = f"""
    <div style="width: 100%; max-width: 800px; height: auto; margin: 0 auto; border: 1px solid #ddd; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); overflow: hidden; font-family: Arial, sans-serif;">
        <div style="padding: 15px; background-color: #f9f9f9; border-bottom: 1px solid #ddd;">
            <h3 style="margin: 0; font-size: 18px; color: #333;">Район: {district_name}</h3>
        </div>
        <div style="padding: 15px; display: flex; flex-direction: column; align-items: center; max-height: 350px; min-width: 300px; overflow-y: auto; background-color: #fff;">
            {chart_html if chart_html else "<p style='color: #888; text-align: center;'>Данные отсутствуют</p>"}
        </div>
    </div>
    """
    return html_template

@lru_cache(maxsize=None)
def process_district_data(district_name):
    match = find_matching_region(district_name, combined_data_df[column_names["region"]].unique())
    if not match:
        return None, 0, "Данные отсутствуют"
    grouped_data = group_columns_by_year(combined_data_df, match)
    charts = []
    population = 0
    for column in grouped_data:
        chart = create_chart(match, grouped_data, column)
        if chart:
            charts.append(chart)
        if column == column_names["total_population"]:
            population = get_closest_year_value(grouped_data, column, datetime.now().year) or 0
    chart_html = ' '.join(charts) if charts else "Данные отсутствуют"
    return match, population, chart_html

# Универсальная функция для получения данных о населённом пункте
def get_settlement_data(settlement, data_type):
    region_name = settlement.get('match')
    if not region_name:
        settlement_name = settlement['name'].lower().replace('ё', 'е')
        region_name = find_matching_region(settlement_name, combined_data_df[column_names["region"]].unique())
    
    if region_name:
        grouped_data = group_columns_by_year(combined_data_df, region_name)
        value = get_closest_year_value(grouped_data, data_type, datetime.now().year)
        return region_name, grouped_data, value
    return None, {}, None

# Создание финальной карты
def create_final_map():
    center = map_parameters["center"]
    zoom = map_parameters["zoom"]
    selected_year = map_parameters["year"]
    selected_marker_type = map_parameters.get("marker_type", "standard")  # Default to standard markers
    
    m = folium.Map(location=center, zoom_start=zoom)
    
    # Добавление слоев базовой карты
    base_maps = {
        "OpenStreetMap": folium.TileLayer("OpenStreetMap"),
        "CartoDB Positron": folium.TileLayer("CartoDB Positron", show=False),
        "CartoDB Dark_Matter": folium.TileLayer("CartoDB Dark_Matter", show=False)
    }
    for name, layer in base_maps.items():
        layer.add_to(m)
    
    # Добавление слоя районов (GeoJSON)
    if map_parameters["map_type"] == "geojson":
        district_layer = folium.FeatureGroup(name="Районы")
        folium.GeoJson(
            geojson_data,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=['district', 'population'], aliases=['Район:', 'Население:'], localize=True),
            popup=folium.GeoJsonPopup(fields=['popup_html'], aliases=[None], localize=True, labels=False, parse_html=True, max_width=800),
            highlight_function=lambda x: {'weight': 3, 'color': 'black'}
        ).add_to(district_layer)
        district_layer.add_to(m)
    
    # Добавление слоя населенных пунктов (OSM)
    elif map_parameters["map_type"] == "osm":
        settlements_data = fetch_settlements_from_overpass()
        if settlements_data:
            settlements = process_settlements_data(settlements_data, combined_data_df)
            for data_type in map_parameters["columns"]:
                layer = folium.FeatureGroup(name=f"{get_russian_name(data_type)} (Населенные пункты)")
                add_markers_based_on_type(settlements, layer, data_type, selected_marker_type)
                layer.add_to(m)
    
    # Добавление контроля слоев
    folium.LayerControl().add_to(m)
    
    # Сохранение карты
    output_map_path = 'final_map.html'
    m.save(output_map_path)
    
    # Предложение загрузить карту на FTP
    ftp_choice = input("Хотите загрузить карту на FTP-сервер? (y/n): ").strip().lower()
    if ftp_choice == 'y':
        ftp_host = input("Введите FTP хост: ").strip()
        ftp_user = input("Введите имя пользователя FTP: ").strip()
        ftp_pass = input("Введите пароль FTP: ").strip()
        ftp_dir = input("Введите директорию FTP (по умолчанию /): ").strip() or "/"
        upload_to_ftp(output_map_path, ftp_host, ftp_user, ftp_pass, ftp_dir)

# Функция для загрузки файла на FTP-сервер
def upload_to_ftp(file_path, ftp_host, ftp_user, ftp_pass, ftp_dir="/"):
    import ftplib
    try:
        # Устанавливаем соединение с FTP-сервером
        with ftplib.FTP(ftp_host) as ftp:
            ftp.login(user=ftp_user, passwd=ftp_pass)
            print(f"Успешно подключено к FTP-серверу: {ftp_host}")
            # Переходим в указанную директорию (по умолчанию корень)
            ftp.cwd(ftp_dir)
            # Открываем файл для загрузки
            with open(file_path, 'rb') as file:
                file_name = os.path.basename(file_path)
                ftp.storbinary(f'STOR {file_name}', file)
                print(f"Файл {file_name} успешно загружен на FTP-сервер в директорию {ftp_dir}")
    except ftplib.all_errors as e:
        print(f"Ошибка при загрузке на FTP: {e}")

# Запуск интерфейса
create_data_source_interface()